In [1]:
import pandas as pd
import numpy as np
import sklearn as sc
import pdpipe as pdp

In [2]:
data = pd.read_csv("USA_Housing.csv")

In [6]:
data.describe()

,Avg. Area Income,Avg. Area House Age,Avg. Area Number of Rooms,Avg. Area Number of Bedrooms,Area Population,Price
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5.000000e+03
mean,68583.108984,5.977222,6.987792,3.981330,36163.516039,1.232073e+06
std,10657.991214,0.991456,1.005833,1.234137,9925.650114,3.531176e+05
min,17796.631190,2.644304,3.236194,2.000000,172.610686,1.593866e+04
25%,61480.562388,5.322283,6.299250,3.140000,29403.928702,9.975771e+05
50%,68804.286404,5.970429,7.002902,4.050000,36199.406689,1.232669e+06
75%,75783.338666,6.650808,7.665871,4.490000,42861.290769,1.471210e+06
max,107701.748378,9.519088,10.759588,6.500000,69621.713378,2.469066e+06


In [7]:
data.head()

,Avg. Area Income,Avg. Area House Age,Avg. Area Number of Rooms,Avg. Area Number of Bedrooms,Area Population,Price,Address
0,79545.458574,5.682861,7.009188,4.09,23086.800503,1.059034e+06,"208 Michael Ferry Apt. 674\nLaurabury, NE 3701..."
1,79248.642455,6.002900,6.730821,3.09,40173.072174,1.505891e+06,"188 Johnson Views Suite 079\nLake Kathleen, CA..."
2,61287.067179,5.865890,8.512727,5.13,36882.159400,1.058988e+06,"9127 Elizabeth Stravenue\nDanieltown, WI 06482..."
3,63345.240046,7.188236,5.586729,3.26,34310.242831,1.260617e+06,USS Barnett\nFPO AP 44820
4,59982.197226,5.040555,7.839388,4.23,26354.109472,6.309435e+05,USNS Raymond\nFPO AE 09386


In [12]:
def sizeOfHouse(n):
    if n<4:
        return "small"
    elif 4<n<=6:
        return "medium"
    else:
        return "big"

In [21]:
data["house_Size"] =  data["Avg. Area Number of Rooms"].apply(sizeOfHouse)
data.head(2)

,Avg. Area Income,Avg. Area House Age,Avg. Area Number of Rooms,Avg. Area Number of Bedrooms,Area Population,Price,Address,sizeOfHouse,house_Size
0,79545.458574,5.682861,7.009188,4.09,23086.800503,1.059034e+06,"208 Michael Ferry Apt. 674\nLaurabury, NE 3701...",big,big
1,79248.642455,6.002900,6.730821,3.09,40173.072174,1.505891e+06,"188 Johnson Views Suite 079\nLake Kathleen, CA...",big,big


In [22]:
#dropping Age column
drop_Age =  pdp.ColDrop("Avg. Area House Age")
data2 = drop_Age(data)
data2.head(1)

,Avg. Area Income,Avg. Area Number of Rooms,Avg. Area Number of Bedrooms,Area Population,Price,Address,sizeOfHouse,house_Size
0,79545.458574,7.009188,4.09,23086.800503,1.059034e+06,"208 Michael Ferry Apt. 674\nLaurabury, NE 3701...",big,big


In [23]:
# Dropping and One Hot Encoding
pipeline  =  pdp.ColDrop("Avg. Area House Age")
pipeline +=  pdp.OneHotEncode("house_Size")
data3     =  pipeline(data)
data3.head(1)

,Avg. Area Income,Avg. Area Number of Rooms,Avg. Area Number of Bedrooms,Area Population,Price,Address,sizeOfHouse,house_Size_medium,house_Size_small
0,79545.458574,7.009188,4.09,23086.800503,1.059034e+06,"208 Michael Ferry Apt. 674\nLaurabury, NE 3701...",big,0,0


In [26]:
#Dropping rows accoding to some value
def price_tag(x):
    if x>250000:
        return "keep"
    else:
        return "drop"

pipeline += pdp.ApplyByCols("Price",price_tag,"Price_tag",drop=False)
pipeline += pdp.ValDrop(["drop"],"Price_tag")
pipeline += pdp.ColDrop("Price_tag")
data4 = pipeline(data)
data4.head(1)

,Avg. Area Income,Avg. Area Number of Rooms,Avg. Area Number of Bedrooms,Area Population,Price,Address,sizeOfHouse,house_Size_medium,house_Size_small
0,79545.458574,7.009188,4.09,23086.800503,1.059034e+06,"208 Michael Ferry Apt. 674\nLaurabury, NE 3701...",big,0,0


In [54]:
#Scaling the data
pipeline_scale = pdp.Scale('StandardScaler',exclude_columns=['house_Size_medium'])
pipeline_updated = pipeline+pipeline_scale
data5     = pipeline_updated(data) 
data5.head(1)

,Avg. Area Income,Avg. Area Number of Rooms,Avg. Area Number of Bedrooms,Area Population,Price,Address,sizeOfHouse,house_Size_medium,house_Size_small
0,1.028113,0.019595,0.087245,-1.32281,-0.500532,"208 Michael Ferry Apt. 674\nLaurabury, NE 3701...",big,-0.443768,-0.024527


In [56]:
def extract_state(token):
    return str(token[-2])

pipeline_tokenize=pdp.TokenizeText('Address')
pipeline_state = pdp.ApplyByCols('Address',extract_state,result_columns='State')
pipeline_state_extract = pipeline_tokenize + pipeline_state
data6 = pipeline_state_extract(data5)
data6.head(1)

,Avg. Area Income,Avg. Area Number of Rooms,Avg. Area Number of Bedrooms,Area Population,Price,State,sizeOfHouse,house_Size_medium,house_Size_small
0,1.028113,0.019595,0.087245,-1.32281,-0.500532,NE,big,-0.443768,-0.024527
